In [24]:
import math
import json
import requests
import itertools
import numpy as np
import time
import praw
import json
import copy
import re
import requests
from datetime import datetime, timedelta

In [18]:

def make_request(uri, max_retries = 5):
    current_try = 1
    def fire_away(uri,current_try):
        if current_try < max_retries:
            try:
                response = requests.get(uri)
                if response.status_code == 200:
                    jsonresp = json.loads(response.content)
                    if jsonresp['metadata']['timed_out'] == False and jsonresp['metadata']['shards']['failed']==0:
                        #and jsonresp['metadata']['shards']['successful']==jsonresp['metadata']['shards']['total']
                        return jsonresp
                    else:
                        current_try+=1
                        time.sleep(1)
                        fire_away(uri,current_try)  
                else:
                    current_try+=1
                    time.sleep(1)
                    fire_away(uri,current_try)
            except:
                current_try+=1
                time.sleep(1)
                fire_away(uri,current_try)
        else:
            print(current_try)
            print('Cannot establish connection or wrong uri',uri)
            return None
    return fire_away(uri,current_try)

#make_request('https://api.pushshift.io/reddit/search/submission/?subreddit=PUBATTLEGROUNDS&user_removed=false&mod_removed=false&&fields=selftext,title,author,id&size=1&before=1544745600&after=1544659200&metadata=true')    

In [ ]:

def run_per_day(subreddit,start,end):
    uri ='https://api.pushshift.io/reddit/search/submission/?subreddit={}&user_removed=false&mod_removed=false&fields=selftext,title,author,id&size=200&after={}&before={}&metadata=true'
    uri=uri.format(subreddit,start,end)
    print('****************************'+str(start))
    response=make_request(uri)
    if response != None :
        submissionsForToplevel = copy.deepcopy(response)
        submissionList = []
        #clean special charachters such as zero width spaces occurs quite frequently
        punc = "1234567890!\"#$%&'()*+,-/:;<=>?@[\]^_`{|}~\n"
        regex = re.compile('[%s]' % re.escape(punc))
        for submission in submissionsForToplevel['data']:
           submission['gencomments'] = makeObj(submission['id'])
           submission['selftext'] = re.sub(r"http\S+", " ", submission['selftext'])
           submission['selftext'] = regex.sub(' ',submission['selftext'])
           if submission['gencomments'] == '' or submission['selftext'] == '[removed]' or submission['selftext'] == '[deleted]' or submission['selftext'].isspace() or submission['selftext'] == '':
                continue
           else:
                submission['gencomments'] = re.sub(r"http\S+", " ", submission['gencomments'])
                submission['gencomments'] = regex.sub(' ',submission['gencomments'])
                titleandtext = submission['title'] + '.' + submission['selftext']
                submission['classifierresults'] = make_request_api(titleandtext)
                submissionList.append(submission)
        return {'jsonA':submissionList}
    else :
        return {'jsonA':None}
        
    
    
def makeIntervals(start,end):
    oneDay = 86400
    startTime = start
    endTime = start + oneDay
    megaTwoLevel = []
    while((startTime - 1) != end):
        dictSubmissions = run_per_day('PUBATTLEGROUNDS',startTime,endTime)
        jsonMA = dictSubmissions.get('jsonA')
        if jsonMA != None:
            megaTwoLevel.append(jsonMA)
        
        startTime = (endTime + 1)
        endTime += oneDay
    
    convJsonA = json.dumps(megaTwoLevel)
    with open('mega2level.json', 'w') as F:
        F.write(convJsonA)
        
#makeIntervals(1544659200,1544745600)    
makeIntervals(1508889600,1540425600)

****************************1508889600


In [3]:
reddit = praw.Reddit(client_id='w44W3IfIXbrE8w',
                     client_secret='j0el6C-wFuaSrMlPeA94aCXRWVQ',
                     redirect_uri='http://localhost:8080',
                     user_agent='testscript by UR')
print(reddit.auth.url(['identity'], '...', 'permanent'))
    

https://www.reddit.com/api/v1/authorize?client_id=w44W3IfIXbrE8w&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


In [4]:
def makeObj(subid):
    submission = reddit.submission(id=subid)
    submission.comments.replace_more(limit=None)
    commentForrest = ''
    for top_level_comment in submission.comments:
        commentForrest += ' ' + top_level_comment.body
        for second_level_comment in top_level_comment.replies:
            commentForrest += ' ' + second_level_comment.body
    #print(type(commentForrest))        
    return commentForrest

def makeCompObj(subid):
    submission = reddit.submission(id=subid)
    submission.comments.replace_more(limit=None)
    commentForrest = ''
    for comment in submission.comments.list():
        commentForrest += ' ' + comment.body
        
    return commentForrest
    



In [ ]:
from string import punctuation
import re

#text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)




punc = "!\"#$%&'()*+,-/:;<=>?@[\]^_`{|}~\n"
regex = re.compile('[%s]' % re.escape(punc))
s='Im a little tpot,short!/\ /"and<> \n\n\nmate fat.'
text = "https://www.dotnetperls.com/punctuation-python matee but this also has ///-/:;<=>?@[\]^_  /n and \n what now"
text = re.sub(r"http\S+", "", text)
print(text)
def strip_punctuation(s):
    return regex.sub('', s)

strip_punctuation(s)


#yourString.isspace()

In [39]:
def make_request_api(data,max_retries = 2):
    #print(data)
    uri = 'http://ec2-18-191-179-70.us-east-2.compute.amazonaws.com:8080/intentclassifier/rest/classify'
    current_try = 1
    def fire_away(uri,current_try):
        if current_try < max_retries:
            try:
                r = requests.post(uri,data={'titleandtext':data})
                if r.status_code == 200:
                    res = r.text
                    return json.loads(res)  
                else:
                    current_try+=1
                    time.sleep(2)
                    fire_away(uri,current_try)
            except:
                current_try+=1
                time.sleep(2)
                fire_away(uri,current_try)
        else:
            print('Cannot establish connection or wrong uri',uri)
            return None
    return fire_away(uri,current_try)

#make_request_api('this game is good.But there are some crashes please fix')

In [ ]:
import json
s = '{"other":"1","inforeq":"0","bug":"1","suggestion":"0","infogive":"0"}'
resss = json.loads(s)
print(resss)